In [ ]:
# Imports
import babypandas as bpd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Animations
from IPython.display import display, IFrame

def show_bootstrapping_slides():
    src = "https://docs.google.com/presentation/d/e/2PACX-1vS_iYHJYXSVMMZ-YQVFwMEFR6EFN3FDSAvaMyUm-YJfLQgRMTHm3vI-wWJJ5999eFJq70nWp2hyItZg/embed?start=false&loop=false&delayms=3000"
    width = 960
    height = 509
    display(IFrame(src, width, height))
    
def show_confidence_interval_slides():
    src="https://docs.google.com/presentation/d/e/2PACX-1vTaPZsueXI6fey_5cj2Y1TevkR1joBvpwaWVsZNvgBlnJSrw1EiBLHJywkFH_QNLU5Tdr6JZgDrhFxG/embed?start=false&loop=false&delayms=3000"
    width = 960
    height = 989
    display(IFrame(src, width, height))

# Lecture 19 – Percentiles and Confidence Intervals

## DSC 10, Winter 2022

### Announcements

- No lecture or discussion on Monday (Presidents' Day).
    - A discussion walkthrough video will be posted instead.
- Homework 5 is due **Saturday 2/19 at 11:59pm**.
- Lab 6 is due **Tuesday 2/22 at 11:59pm**.
- Sign up to take my [History of Data Science](https://historyofdsc.com) (DSC 90) seminar course next quarter!
    - 2 units, ~2 hours a week of class.

### Agenda

- Review: bootstrapping.
- Percentiles.
- Confidence intervals.

## Review: bootstrapping

**Goal:** Estimate the distribution of a sample statistic, given just a single sample.

In [ ]:
show_bootstrapping_slides()

### Recall: City of San Diego salary data

- We're interested in the median of all salaries (this is our population parameter).
- In this case we know it, but in reality we will not.

In [ ]:
population = bpd.read_csv('data/2020_salaries.csv')
population_median = population.get('TotalWages').median()
population_median

### Taking a single (large) random sample

In [ ]:
np.random.seed(23) # Ignore this
my_sample = population.sample(500, replace=False)
my_sample

### Bootstrapping

- Step 0: Collect a sample of the population – done.
- Step 1: Resample from the sample, **with replacement**, to create a new bootstrapped resample of the same size.
- Step 2: Compute the statistic on this resample and store it in an array.
- Step 3: Repeat Steps 1 and 2 many, many times.

In [ ]:
# Note that the population DataFrame doesn't appear anywhere here!
n_resamples = 5000
boot_medians = np.array([])

for i in np.arange(n_resamples):
    
    # Resample from my_sample
    resample = my_sample.sample(500, replace=True)
    
    # Compute the median
    median = resample.get('TotalWages').median()
    
    # Store it in our array of medians
    boot_medians = np.append(boot_medians, median)

In [ ]:
boot_medians

Now that we have this array of bootstrapped sample medians, we can visualize it.

In [ ]:
bpd.DataFrame().assign(BootstrapMedians=boot_medians).plot(kind='hist', density=True, bins=np.arange(60000, 85000, 1000), ec='w', figsize=(10, 5))
plt.scatter(population_median, 0.000004, color='orange', s=100, label='population median').set_zorder(2)
plt.legend();

- Again, we can only plot the orange dot because we happen to know the population parameter in this case (for teaching purposes).
    - In reality, you will not know the population parameter!

### Example: estimating the max of a population

Suppose we want to estimate the maximum salary of all San Diego city employees, given just a single sample `my_sample`.

In [ ]:
# The true maximum salary in the population
population_max = population.get('TotalWages').max()
population_max

### Running the bootstrap

- We want to estimate the maximum number in the population.
- Our estimate will be the max in the sample.
- We run the bootstrap:

In [ ]:
n_resamples = 5000
boot_maxes = np.array([])

for i in np.arange(n_resamples):

    resample = my_sample.sample(my_sample.shape[0], replace=True)
    
    boot_max = resample.get('TotalWages').max()
    
    boot_maxes = np.append(boot_maxes, boot_max)

In [ ]:
boot_maxes

### Visualize

The bootstrap distribution doesn't surround the population maximum (orange dot) of 320699. Why not? 🤔

In [ ]:
bpd.DataFrame().assign(BootstrapMax=boot_maxes).plot(kind='hist', 
                                                     density=True, 
                                                     bins=np.arange(180000, 325000, 20000), 
                                                     ec='w',
                                                     figsize=(10, 5))
plt.scatter(population_max, 0.0000008, color='orange', s=100, label='population max')
plt.legend();

### The answer

- The largest value in our original sample was 257062. 
- Therefore, the largest value in any bootstrapped sample is at most 257062.
- Generally, the bootstrap works better for measures of central tendency or variation (means, medians, variances) than it does for extremas (maxes and mins).
    - If we instead repeatedly sampled from the population directly, there would be more possibilities for the largest possible value in each sample, and so the distribution of the sample max would not have the "gaps" in it that the distribution on the previous slide does.

In [ ]:
my_sample.get('TotalWages').max()

## Percentiles

### What's the point?

- We just learned how to approximate the distribution of a sample statistic.
- This means that we now have a sense of how much our estimates can vary.
- If someone asks, "what's your best guess of the population parameter?"
    - We're not just going to give them a single sample statistic.
    - We're not going to give them our entire bootstrapped distribution, either.
    - Instead, we may want to give them a range, or **interval**, of likely values for the population parameter.
- In order to talk about intervals, we need to talk about percentiles.

### Mathematical definition

> Let $p$ be a number between 0 and 100. The $p$th percentile of a collection is the smallest value in the collection that is *at least as large* as $p$% of all the values. 

By this definition, any percentile between 0 and 100 can be computed for any collection of values and is always an element of the collection. 

### How to calculate percentiles using mathematical definition

Suppose there are $n$ elements in the collection. To find the $p$th percentile:

1. Sort the collection in increasing order.
2. Define $h$ to be $p\%$ of $n$: 

$$h = \frac p{100} \cdot n$$

3. If $h$ is an integer, define $k = h$. Otherwise, let $k$ be the smallest integer greater than $h$.

4. Take the $k$th element of the sorted collection (start counting from 1, not 0).

### Example

What is the 25th percentile of the array `np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])`?

### Discussion Question

What is the 60th percentile of this sorted array?

`np.array([10, 18, 35, 45, 55, 56, 70, 89])`

- A. 45
- B. 53
- C. 55
- D. 60
- E. 70

### To answer, go to [menti.com](https://menti.com) and enter the code 5474 0323.

### Calculating the percentile using our mathematical definition

In [ ]:
def our_percentile(data, p):
    n = len(data)
    h = (p / 100) * n
    k = int(np.ceil(h)) # If h is an integer, this is h. Otherwise, rounds up.
    return data[k - 1] # -1 because Python is 0-indexed, but regular math is 1-indexed.

In [ ]:
our_percentile(np.arange(1, 11), 25)

### Another definition of percentile

- The `numpy` package provides a function to calculate percentiles, `np.percentile(array, p)`, which returns the `p`th percentile of `array`.
- The `numpy` method of calculating percentile is slightly different than ours.
    - The result doesn't have to be an element of the array (unlike in our mathematical definition). 
    - Details are unimportant, just know that our mathematical definition does not always match `np.percentile`!

In [ ]:
np.percentile(np.arange(1, 11), 25)

## Confidence intervals

### Confidence intervals


- Goal: estimate an unknown population parameter.
- We have been saying 
>"My guess for the population parameter is my sample statistic, $x$."
- We want to say 
>"My guess is that the population parameter is between $a$ and $b$."
- Let's use the bootstrapped distribution of an estimate to compute an **interval** that contains "the bulk" of the estimates.

### A 95% confidence interval for median salary

- Recall our bootstrapped distribution of the sample median.
- Suppose by "bulk", we mean containing the middle 95% of the area.

In [ ]:
bpd.DataFrame().assign(BootstrapMedians=boot_medians).plot(kind='hist', density=True, bins=np.arange(60000, 85000, 1000), ec='w', figsize=(10, 5))
plt.scatter(population_median, 0.000004, color='orange', s=100, label='population median').set_zorder(2)
plt.legend();

### Finding endpoints

- We want to find two points, $x$ and $y$, such that the area:
    - to the left of $x$ is about 2.5%.
    - to the right of $y$ is about 2.5%.
- The interval $[x,y]$ will contain about 95% of the total area, i.e. **95% of the total values**.
- $x$ and $y$ are the 2.5th percentile and 97.5th percentile, respectively.

In [ ]:
# Left endpoint
left = np.percentile(boot_medians, 2.5)
left

In [ ]:
# Right endpoint
right = np.percentile(boot_medians, 97.5)
right

In [ ]:
# Therefore, our interval is
[left, right]

You will use the code above **very** frequently moving forward!

### Visualizing our 95% confidence interval

- Let's draw the interval we just computed on the histogram.
- 95% of the bootstrap medians fell into this interval.

In [ ]:
bpd.DataFrame().assign(BootstrapMedians=boot_medians).plot(kind='hist', density=True, bins=np.arange(60000, 85000, 1000), ec='w', figsize=(10, 5))
plt.plot([left, right], [0, 0], color='lime', linewidth=12, label='95% confidence interval');
plt.scatter(population_median, 0.000004, color='orange', s=100, label='population median').set_zorder(3)
plt.legend();

- In this case, our 95% confidence interval (lime green) contains the true population parameter (orange dot).
    - It won't always, because you might have a bad original sample!
    - In reality, you won't know where the population parameter is, and so you won't know if your confidence interval contains it.

### Discussion Question

Would an 80% confidence interval be wider, narrower, or the same size as our 95% confidence interval?

- A. Wider
- B. Narrower
- C. The same size

### To answer, go to [menti.com](https://menti.com) and enter the code 5474 0323.

## Interpreting confidence intervals

### Interpreting confidence intervals

- 95% of our bootstrap medians fell within this interval.
- We're pretty confident that the true median does, too.
- How confident should we be about this? **What does a 95% confidence interval mean?**

### Capturing the true value

- Consider the process of:
    - Collecting a **new original sample** from the population,
    - Bootstrap resampling from it many times, computing the statistic (e.g. median) in each resample, and
    - Constructing a new 95% confidence interval.
- What a 95% confidence level states is that **approximately 95% of the time, the intervals you create through this process will contain the true population parameter**.
- The confidence is in **the process** that generates the interval.

### Many confidence intervals

- We repeated the process outlined on the previous slide 200 times, to come up with 200 confidence intervals.
    - We did this in advance and saved the results to a file to save time.
- The resulting CIs are stored in the array `many_cis` below.

In [ ]:
many_cis = np.load('data/many_cis.npy')
many_cis

In the visualization below,
- The orange line represents the population parameter. **This is not random!**
- Each green line represents a separate confidence interval, created using the specified process.
- Most of these confidence intervals contain the true parameter – but not all!

In [ ]:
plt.figure(figsize=(10, 5))
for i, ci in enumerate(many_cis):
    plt.plot([ci[0], ci[1]], [i, i], color='green', linewidth=1)
plt.axvline(x=population_median, color='orange');

### Which confidence intervals don't contain the true parameter?

In [ ]:
plt.figure(figsize=(10, 5))
count_outside = 0
for i, ci in enumerate(many_cis):
    if ci[0] > population_median or ci[1] < population_median:
        plt.plot([ci[0], ci[1]], [i, i], color='green', linewidth=1)
        count_outside = count_outside + 1
plt.axvline(x=population_median, color='orange');

In [ ]:
count_outside

- 13 of my 200 confidence intervals didn't contain the true parameter.
- That means 187/200, or 93.5% of them, did.
    - This is pretty close to 95%!
- In reality, you will have **no way of knowing if your confidence interval contains the true parameter**, since you have no way of knowing if your original sample is good.

### Misinterpreting confidence intervals

In [ ]:
# Our interval
[left, right]

- It is **not true** that this interval contains 95% of all salaries.
- Instead, it is true that this interval contains 95% of all bootstrapped **median** salaries.

In [ ]:
population.plot(kind='hist', y='TotalWages', density=True, ec='w', figsize=(10, 5))
plt.plot([left, right], [0, 0], color='lime', linewidth=10, label='95% confidence interval');
plt.legend();

### Misinterpreting confidence intervals

In [ ]:
# Our interval
[left, right]

- It is **not true** that there is a 95% chance that this interval contains the population parameter.
    - The population parameter is fixed. In reality, we might not know it, but it is **not random**.
    - The interval above is not random, either (but the process that creates it is).
    - Either the population parameter is in the interval, or it is not. There is no randomness involved.
- Remember that the 95% confidence is in the process that created an interval.
    - This process (sampling, then bootstrapping, then creating an interval) creates a good interval roughly 95% of the time.

In [ ]:
show_confidence_interval_slides()

## Summary

### Summary

- The $p$th percentile of a collection is the smallest value in the collection that is *at least as large* as $p$% of all the values. 
- After creating a bootstrapped distribution of a statistic, we can create a $c$% confidence interval by taking the middle $c$% of values of the bootstrapped distribution.
    - Interpretation: if you create many different c% confidence intervals, roughly c% of them will contain the true population parameter.
    - This method of creating confidence intervals is known as the **bootstrap percentile method**.
- **Next time:** We'll look at how to use confidence intervals for hypothesis tests where the null hypothesis is that the population parameter is equal to some value. We'll also start discussing center and spread (mean, median, and variance).